In [1]:
import sys
sys.path.append("/home/hacene/Documents/StyleGAN-pytorch-lightining/src")

In [2]:
import torch
import pytorch_lightning as pl

In [3]:
from models import StyleGAN
from callbacks import UpdateBatchSizeDataLoader, UpdateMixingDepth

import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import (
    ToTensor,
    Normalize,
    Compose,
    Resize,
    RandomHorizontalFlip,
)


In [4]:
def get_transform(new_size=None):
    """
    obtain the image transforms required for the input data
    :param new_size: size of the resized images
    :return: image_transform => transform object from TorchVision
    """

    if new_size is not None:
        image_transform = Compose(
            [
                RandomHorizontalFlip(),
                Resize(new_size),
                ToTensor(),
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
            ]
        )

    else:
        image_transform = Compose(
            [
                RandomHorizontalFlip(),
                ToTensor(),
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
            ]
        )
    return image_transform

In [5]:
class DataModule(pl.LightningDataModule):
    def __init__(self, data_dir="..", batch_size=64):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage):
        transform = get_transform()
        self.dataset = CIFAR10(root=".", train=False, download=True, transform=transform)

    def train_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.batch_size, shuffle=True)

In [6]:
dataloader = DataModule()

In [7]:
opt_args = {"lr": 0.003, "betas": (0, 0.99), "eps": 1e-8}
model = StyleGAN(
    structure="linear",
    resolution=32,
    num_channels=3,
    z_latent_dim=512,
    generator_args={},
    discriminator_args={},
    generator_opt_args=opt_args,
    discriminator_opt_args=opt_args,
    start_depth=0,
)

In [8]:
callback = UpdateMixingDepth(epochs_for_each_depth=[2, 2, 2, 2], fade_for_each_depth=[50, 50, 50, 50])

In [9]:
trainer = pl.Trainer(max_epochs=8, callbacks=[callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, dataloader)